In [1]:
import os
import numpy as np 
import pandas as pd 

In [2]:
!wget -P ./../../../dataset/ -nc https://cleanlab-public.s3.amazonaws.com/RegressionBenchmark/airquality_co.csv

--2022-12-07 13:09:18--  https://cleanlab-public.s3.amazonaws.com/RegressionBenchmark/airquality_co.csv
Resolving cleanlab-public.s3.amazonaws.com (cleanlab-public.s3.amazonaws.com)... 52.217.130.249, 52.216.101.99, 52.217.17.244, ...
Connecting to cleanlab-public.s3.amazonaws.com (cleanlab-public.s3.amazonaws.com)|52.217.130.249|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 575572 (562K) [text/csv]
Saving to: './../../../dataset/airquality_co.csv'

airquality_co.csv   100%[===================>] 562.08K   837KB/s    in 0.7s    

2022-12-07 13:09:19 (837 KB/s) - './../../../dataset/airquality_co.csv' saved [575572/575572]



In [3]:
path = "./../../../dataset/airquality_co.csv"
dataset_name = os.path.splitext(os.path.basename(path))[0]
data = pd.read_csv(path, index_col=0)
print("shape of data: ", data.shape)
data.head() 

shape of data:  (7344, 8)


,time_slot,day_of_week,T,RH,AH,true_label,given_label,true_error
0,evening,wednesday,13.6,48.9,0.7578,0.327392,1.140560,0
1,evening,wednesday,13.3,47.7,0.7255,-0.090299,0.829606,0
2,evening,wednesday,11.9,54.0,0.7502,0.048931,1.332621,1
3,evening,wednesday,11.0,60.0,0.7867,0.048931,1.213726,1
4,night,wednesday,11.2,59.6,0.7888,-0.368759,0.738148,1


In [4]:
training_data = data.drop(['true_label', 'true_error'], axis=1)
training_data.head(2)

,time_slot,day_of_week,T,RH,AH,given_label
0,evening,wednesday,13.6,48.9,0.7578,1.140560
1,evening,wednesday,13.3,47.7,0.7255,0.829606


In [5]:
y = training_data['given_label']
X = training_data.drop(['given_label'], axis=1)

###### Add your training code here ###############
# model = 
# model_fit = model.fit(X,y)
# predictions = model.predict(X)

##### saving the predictions ####################
# savepath = "./../predictions/model_predictions.npy"
# np.save(savepath, np.array(predictions))

## Section section is relevant only if using AutoGluon

Uncomment the code in following cells and run to generate output and save to predictions folder. 

### set up arguments 

In [6]:
from autogluon.tabular import TabularPredictor

data_without_label = training_data.drop(['given_label'], axis=1)
savepath = "./../trained_models/"
if not os.path.isdir(savepath):
    os.mkdir(savepath)

predictorArgs = {
                    "path"             : savepath, 
                    "label"            : 'given_label',
                    "eval_metric"      : 'r2',
                    "problem_type"     : 'regression'
                }

hyperparameters = { 
                    'GBM'   : {}, 
                    'FASTAI': {}, 
                    'RF'    : {'criterion': 'squared_error', 
                                'ag_args': {'name_suffix': 'MSE', 
                                'problem_types': ['regression', 'quantile']}}
                    }

### Normal Model fitting

In [7]:
predictor = TabularPredictor(**predictorArgs)
predictor.fit(training_data, hyperparameters=hyperparameters)
leaderboard = predictor.leaderboard(training_data, silent=True)
leaderboard

Beginning AutoGluon training ...
AutoGluon will save models to "./../trained_models/"
AutoGluon Version:  0.5.2
Python Version:     3.9.15
Operating System:   Darwin
Train Data Rows:    7344
Train Data Columns: 5
Label Column: given_label
Preprocessing data ...
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    1299.94 MB
	Train Data (Original)  Memory Usage: 1.12 MB (0.1% of available memory)
	Inferring data type of each feature based on column values. Set feature_metadata_in to manually specify special dtypes of the features.
	Stage 1 Generators:
		Fitting AsTypeFeatureGenerator...
	Stage 2 Generators:
		Fitting FillNaFeatureGenerator...
	Stage 3 Generators:
		Fitting IdentityFeatureGenerator...
		Fitting CategoryFeatureGenerator...
			Fitting CategoryMemoryMinimizeFeatureGenerator...
	Stage 4 Generators:
		Fitting DropUniqueFeatureGenerator...
	Types of features in original data (raw dtype, special dtyp

,model,score_test,score_val,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,RandomForestMSE,0.869871,0.418549,0.219847,0.039326,1.193050,0.219847,0.039326,1.193050,1,True,2
1,WeightedEnsemble_L2,0.744934,0.445698,0.278591,0.044666,3.846245,0.033383,0.000600,0.028571,2,True,4
2,LightGBM,0.580057,0.428501,0.025361,0.004740,2.624624,0.025361,0.004740,2.624624,1,True,1
3,NeuralNetFastAI,0.418339,0.375915,0.047207,0.011256,4.654114,0.047207,0.011256,4.654114,1,True,3


### Cross Validation fitting

In [8]:
predictor_cv = TabularPredictor(**predictorArgs) 
predictor_cv.fit(training_data, presets="best_quality", num_stack_levels= 0, hyperparameters=hyperparameters)
leaderboard_cv = predictor_cv.leaderboard(training_data, silent=True)
leaderboard_cv

Presets specified: ['best_quality']
Stack configuration (auto_stack=True): num_stack_levels=0, num_bag_folds=8, num_bag_sets=1
Beginning AutoGluon training ...
AutoGluon will save models to "./../trained_models/"
AutoGluon Version:  0.5.2
Python Version:     3.9.15
Operating System:   Darwin
Train Data Rows:    7344
Train Data Columns: 5
Label Column: given_label
Preprocessing data ...
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    1498.27 MB
	Train Data (Original)  Memory Usage: 1.12 MB (0.1% of available memory)
	Inferring data type of each feature based on column values. Set feature_metadata_in to manually specify special dtypes of the features.
	Stage 1 Generators:
		Fitting AsTypeFeatureGenerator...
	Stage 2 Generators:
		Fitting FillNaFeatureGenerator...
	Stage 3 Generators:
		Fitting IdentityFeatureGenerator...
		Fitting CategoryFeatureGenerator...
			Fitting CategoryMemoryMinimizeFeatureGenerat

,model,score_test,score_val,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,RandomForestMSE_BAG_L1,0.922527,0.43087,0.255900,0.188446,0.907120,0.255900,0.188446,0.907120,1,True,2
1,WeightedEnsemble_L2,0.808978,0.44857,0.914207,0.314260,45.057158,0.002881,0.000186,0.055887,2,True,4
2,LightGBM_BAG_L1,0.639082,0.41126,0.287858,0.056677,14.893195,0.287858,0.056677,14.893195,1,True,1
3,NeuralNetFastAI_BAG_L1,0.429583,0.38826,0.367569,0.068952,29.200956,0.367569,0.068952,29.200956,1,True,3


### predicting and saving arrays 

In [ ]:
models = {  'normal'            : list(leaderboard['model']), 
            'crossValidation'   : list(np.append(leaderboard_cv['model'], ['oof']))}

predictors = { 'normal'          : predictor, 
               'crossValidation' : predictor_cv}

# check if path is available, else create it. 
pred_path = "./../predictions/"
if not os.path.isdir(pred_path):
    os.mkdir(pred_path)

for model_type in models.keys():
    if model_type == "normal": 
        for model in models[model_type]:
            savepath = os.path.join(pred_path, model+".npy")
            predictions = predictors[model_type].predict(data_without_label, model=model)
            np.save(savepath, np.array(predictions))
    elif model_type == "crossValidation":
        for model in models[model_type]:
            if model == "oof":
                predictions = predictors[model_type].get_oof_pred()
                savepath = os.path.join(pred_path, model+".npy")
            elif model in models['normal']:
                predictions = predictors[model_type].predict(data_without_label, model=model)
                savepath = os.path.join(pred_path, model+"_CV.npy")
            else: 
                predictions = predictors[model_type].predict(data_without_label, model=model)
                savepath = os.path.join(pred_path, model+".npy")

            np.save(savepath, np.array(predictions))